## Requisitos
pip install databricks-sql-connector<br>
pip install databricks-sql-connector openai<br>
pip install databricks-sql-connector[pyarrow]<br>
pip install pandas


In [ ]:
hostname_warehouse = "dbc-17f1c94d-f9f3.cloud.databricks.com"
path_http = "/sql/1.0/warehouses/629b6c5c9e7bd645"
DATABRICKS_API_KEY = "sua databricks api key aqui"
OPENAI_API_KEY = "sua openai api key aqui"


In [41]:
## Importar bibliotecas
from databricks import sql
from openai import OpenAI


In [ ]:
# Exemplo de consulta com query ao Databricks
# e conversão para DataFrame Pandas
server_hostname = hostname_warehouse
http_path = path_http
access_token = DATABRICKS_API_KEY

try:
    with sql.connect(server_hostname=server_hostname,
                     http_path=http_path,
                     access_token=access_token,
                     use_cloud_fetch=True) as connection:

        with connection.cursor() as cursor:
            query = '''
                    SELECT transactionID, product, quantity, unitPrice
                    FROM samples.bakehouse.sales_transactions
                    WHERE transactionID = 2003017
                    '''
            cursor.execute(query)
            arrow_table = cursor.fetchall_arrow()
            df = arrow_table.to_pandas()
    print(df)
except Exception as e:
    print(f"Ocorreu um erro: {e}")


   transactionID                 product  quantity  unitPrice
0        2003017  Austin Almond Biscotti        40          3


In [ ]:
# Exemplo de consulta em linguagem natural ao Databricks
# com conversão para DataFrame Pandas

# Configuração do Databricks
server_hostname = hostname_warehouse
http_path = path_http
databricks_token = DATABRICKS_API_KEY

# Configuração do OpenAI
openai_api_key = OPENAI_API_KEY
client = OpenAI(api_key=openai_api_key)

# Esquema da tabela para o LLM
table_schema = """
A tabela samples.bakehouse.sales_transactions contém dados de transações.
As colunas incluem:
- transactionId (integer)
- product (string)
- quantity (decimal)
- unitPrice (decimal)
"""

def generate_sql_from_natural_language(question: str) -> str:
    prompt = f"""
            Dado o seguinte esquema de tabela:
            {table_schema}
            Converta a seguinte pergunta para uma consulta SQL válida para 
            a tabela `samples`.`bakehouse`.`sales_transactions`:
            Pergunta: {question}
            Retorne apenas a query da consulta SQL sem explicações,
            no seguinte formato:
            SELECT col1, col2 FROM table WHERE condition,
            ou apenas a query se não houver condição.;
            """
    response = client.completions.create(
        model="gpt-4o-mini",
        prompt=prompt,
        max_tokens=100
    )
    generated_sql = response.choices[0].text.strip()
    return generated_sql

def execute_natural_language_query(question: str):
    sql_query = generate_sql_from_natural_language(question)
    try:
        with sql.connect(server_hostname=server_hostname,
                         http_path=http_path,
                         access_token=databricks_token,
                         use_cloud_fetch=True) as connection:

            with connection.cursor() as cursor:
                cursor.execute(sql_query)
                arrow_table = cursor.fetchall_arrow()
                df = arrow_table.to_pandas()
        print(df)
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

# Exemplo de uso
pergunta = "quero a linha que tenha o id da transação 2003017, " \
"e retorne apenas as colunas transactionID, product, quantity, unitPrice"
execute_natural_language_query(pergunta)

SQL gerado: SELECT transactionId, product, quantity, unitPrice FROM samples.bakehouse.sales_transactions WHERE transactionId = 2003017;

   transactionId                 product  quantity  unitPrice
0        2003017  Austin Almond Biscotti        40          3
